In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *

spark = SparkSession.builder.getOrCreate()

path = "/mnt/c/Ambarish/UsedCars/input/vehicles.csv"

21/11/26 15:10:28 WARN Utils: Your hostname, DESKTOP-OR4MSG8 resolves to a loopback address: 127.0.1.1; using 172.18.224.90 instead (on interface eth0)
21/11/26 15:10:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/11/26 15:10:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# define schema for our data
schema = StructType([
   StructField("Id", IntegerType(), False),
   StructField("url", StringType(), False),
   StructField("region", StringType(), False),
   StructField("region_url", StringType(), False),
   StructField("price", LongType(), False),
   StructField("year", IntegerType(), False),
   StructField("manufacturer", StringType(), False),
   StructField("model", StringType(), False),
   StructField("condition", StringType(), False),
   StructField("cylinders", StringType(), False),
   StructField("fuel", StringType(), False),
   StructField("odometer", StringType(), False),
   StructField("title_status", StringType(), False),
   StructField("transmission", StringType(), False),
   StructField("VIN", StringType(), False),
   StructField("drive", StringType(), False),
   StructField("size", StringType(), False),
   StructField("type", StringType(), False),
   StructField("paint_color", StringType(), False),
   StructField("image_url", StringType(), False),
   StructField("description", StringType(), False),
   StructField("county", StringType(), False),
   StructField("state", StringType(), False),
   StructField("lat", StringType(), False),
   StructField("long", StringType(), False),
   StructField("posting_date", StringType(), False)]
   )

In [3]:
df = spark.read.load(path,
                     format="csv", 
                     sep=",", 
                     schema =schema, 
                     header="true")

In [4]:
df.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- url: string (nullable = true)
 |-- region: string (nullable = true)
 |-- region_url: string (nullable = true)
 |-- price: long (nullable = true)
 |-- year: integer (nullable = true)
 |-- manufacturer: string (nullable = true)
 |-- model: string (nullable = true)
 |-- condition: string (nullable = true)
 |-- cylinders: string (nullable = true)
 |-- fuel: string (nullable = true)
 |-- odometer: string (nullable = true)
 |-- title_status: string (nullable = true)
 |-- transmission: string (nullable = true)
 |-- VIN: string (nullable = true)
 |-- drive: string (nullable = true)
 |-- size: string (nullable = true)
 |-- type: string (nullable = true)
 |-- paint_color: string (nullable = true)
 |-- image_url: string (nullable = true)
 |-- description: string (nullable = true)
 |-- county: string (nullable = true)
 |-- state: string (nullable = true)
 |-- lat: string (nullable = true)
 |-- long: string (nullable = true)
 |-- posting_date: string (nu

In [5]:
len(df.columns)

26

In [6]:
df.show(3)

21/11/26 15:10:34 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----+--------------------+------------+--------------------+-----+----+------------+-----+---------+---------+----+--------+------------+------------+----+-----+----+----+-----------+---------+-----------+------+-----+----+----+------------+
|  Id|                 url|      region|          region_url|price|year|manufacturer|model|condition|cylinders|fuel|odometer|title_status|transmission| VIN|drive|size|type|paint_color|image_url|description|county|state| lat|long|posting_date|
+----+--------------------+------------+--------------------+-----+----+------------+-----+---------+---------+----+--------+------------+------------+----+-----+----+----+-----------+---------+-----------+------+-----+----+----+------------+
|null|https://prescott....|    prescott|https://prescott....| 6000|null|        null| null|     null|     null|null|    null|        null|        null|null| null|null|null|       null|     null|       null|  null|   az|null|null|        null|
|null|https://fayar.cra...|f

In [7]:
df.select("cylinders").show(10)

+---------+
|cylinders|
+---------+
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
+---------+
only showing top 10 rows



In [8]:
from pyspark.sql import functions

split_col = functions.split(df['cylinders'], ' ')
df3 = df.withColumn('cylinders', split_col.getItem(0))
df3.select("cylinders").show()

+---------+
|cylinders|
+---------+
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
+---------+
only showing top 20 rows



# Casting into proper data types

# Clean the data

In [9]:
df = df.filter(df.price > 0)

# Summary of Price of Used Car

In [10]:
df.describe(['price']).show()

+-------+--------------------+
|summary|               price|
+-------+--------------------+
|  count|              393985|
|   mean|   81477.62804929122|
| stddev|1.2680636885188837E7|
|    min|                   1|
|    max|          3736928711|
+-------+--------------------+



In [11]:
df.createOrReplaceTempView("used_cars")

# Count of Cars where there is an Invalid year

In [12]:
sql_query = "SELECT count(1) \
FROM used_cars \
WHERE year < 0"

sqlDF = spark.sql(sql_query)
sqlDF.show(25,truncate = False)

+--------+
|count(1)|
+--------+
|0       |
+--------+



# Count of Cars of VINS appearing more than Once

In [13]:
sql_query = "SELECT VIN , COUNT(1) as CountVIN \
FROM used_cars \
GROUP BY VIN HAVING CountVIN > 1"

sqlDF = spark.sql(sql_query)
sqlDF.show(25,truncate = False)

+-----------------+--------+
|VIN              |CountVIN|
+-----------------+--------+
|WBALM1C55BE393513|97      |
|ZARFAEDN0J7565017|24      |
|KNDJ23AU8L7109397|2       |
|2T2ZK1BA8DC129687|3       |
|ZFBERFAT9F6A76965|3       |
|ZARFAEEN3H7558717|2       |
|1FDAX57P26ED52391|2       |
|3GNCJNSB5LL189135|3       |
|1GT120E88FF513709|9       |
|1GYS3CEF5DR103313|2       |
|1FADP5AU7DL503286|2       |
|WBA8K3C56JA023782|3       |
|WAUR4AF53JA008266|41      |
|1FT7W2BT7KED12158|4       |
|1FTYR1ZM0JKA98201|3       |
|1G6AR5SS8J0137892|27      |
|ZFBHRFAB0M6T14104|3       |
|1N4AZ0CPXDC415680|2       |
|JTEBU5JR2F5271740|2       |
|1FTBF2A67BED05863|3       |
|4T1BK1EB1DU028175|4       |
|JTDKN3DU8C5391950|2       |
|232709           |2       |
|KNMAT2MT7GP679827|8       |
|1J4FA49S11P364952|4       |
+-----------------+--------+
only showing top 25 rows



# Count of Cars having same URL more than Once

In [14]:
sql_query = "SELECT url , COUNT(1) as CountURL \
FROM used_cars \
GROUP BY url  HAVING CountURL > 1"

sqlDF = spark.sql(sql_query)
sqlDF.show(25,truncate = False)

+---+--------+
|url|CountURL|
+---+--------+
+---+--------+



# Cars by Region

In [15]:
sql_query = "SELECT region,COUNT(1) as COUNTREGION FROM used_cars  \
GROUP BY region ORDER BY COUNTREGION DESC"

sqlDF = spark.sql(sql_query)
sqlDF.show(25,truncate = False)

+-----------------------+-----------+
|region                 |COUNTREGION|
+-----------------------+-----------+
|columbus               |3392       |
|jacksonville           |3177       |
|omaha / council bluffs |2967       |
|new hampshire          |2944       |
|grand rapids           |2934       |
|milwaukee              |2917       |
|central NJ             |2905       |
|tampa bay area         |2894       |
|sarasota-bradenton     |2882       |
|kansas city, MO        |2876       |
|nashville              |2867       |
|las vegas              |2865       |
|boston                 |2856       |
|detroit metro          |2854       |
|orlando                |2851       |
|chicago                |2851       |
|south florida          |2848       |
|ft myers / SW florida  |2840       |
|denver                 |2821       |
|st louis, MO           |2812       |
|spokane / coeur d'alene|2807       |
|north jersey           |2802       |
|south jersey           |2799       |
|cleveland  

#  Cars by Model

In [16]:
sql_query = "SELECT model,COUNT(1) \
FROM used_cars \
GROUP BY model \
ORDER BY COUNT(1) DESC"


sqlDF = spark.sql(sql_query)
sqlDF.show(25,truncate = False)

+--------------+--------+
|model         |count(1)|
+--------------+--------+
|f-150         |7116    |
|null          |4701    |
|silverado 1500|4546    |
|1500          |3803    |
|camry         |2827    |
|silverado     |2780    |
|accord        |2749    |
|wrangler      |2674    |
|civic         |2597    |
|escape        |2517    |
|altima        |2491    |
|2500          |2467    |
|tacoma        |2378    |
|explorer      |2252    |
|grand cherokee|2229    |
|mustang       |2019    |
|corolla       |1973    |
|equinox       |1769    |
|cr-v          |1722    |
|fusion        |1670    |
|focus         |1669    |
|corvette      |1617    |
|malibu        |1577    |
|tahoe         |1524    |
|rav4          |1520    |
+--------------+--------+
only showing top 25 rows




[Stage 22:==================================================>     (10 + 1) / 11]



#  Calculate Average Price , Standard Deviation for Each Condition

In [17]:
sql_query = "SELECT condition,COUNT(1),AVG(price) as Average, \
STDDEV(price) as StandardDev, \
MIN(price) as MIN, \
MAX(price) as MAX \
FROM used_cars \
GROUP BY condition \
ORDER BY COUNT(1) DESC"


sqlDF = spark.sql(sql_query)
sqlDF.show(25,truncate = False)

+---------+--------+------------------+--------------------+---+----------+
|condition|count(1)|Average           |StandardDev         |MIN|MAX       |
+---------+--------+------------------+--------------------+---+----------+
|null     |151389  |112177.00333577736|1.4667367591470314E7|1  |3024942282|
|good     |119155  |33173.68291720868 |3259849.8200198743  |1  |1111111111|
|excellent|94925   |54885.52424545694 |1.2128943588720001E7|1  |3736928711|
|like new |20060   |38430.82976071785 |1558111.0473921574  |1  |135008900 |
|fair     |6729    |765614.2440184277 |4.7999096711095154E7|1  |3736928711|
|new      |1132    |27272.732332155476|27116.586017930298  |1  |182000    |
|salvage  |595     |3641.8924369747897|4715.988233164577   |1  |35989     |
+---------+--------+------------------+--------------------+---+----------+



# Count per Group : Model , Manufacturer and Year

In [18]:
sql_query = "SELECT model,manufacturer,year,COUNT(1) \
FROM used_cars \
GROUP BY model,manufacturer,year \
ORDER BY COUNT(1) DESC"


sqlDF = spark.sql(sql_query)
sqlDF.show(25,truncate = False)

+--------------------------+------------+----+--------+
|model                     |manufacturer|year|count(1)|
+--------------------------+------------+----+--------+
|f-150                     |ford        |2018|757     |
|f-150                     |ford        |2013|754     |
|Scion iM Hatchback 4D     |null        |2016|640     |
|f-150                     |ford        |2014|621     |
|f-150                     |ford        |2017|571     |
|f-150                     |ford        |2016|548     |
|f-150                     |ford        |2015|542     |
|silverado 1500            |chevrolet   |2015|524     |
|silverado 1500            |chevrolet   |2014|509     |
|1500                      |ram         |2017|464     |
|f-150                     |ford        |2010|434     |
|1500                      |ram         |2014|410     |
|altima                    |nissan      |2013|405     |
|1500                      |ram         |2016|400     |
|1500 classic regular cab  |ram         |2019|38

# Count per Group : Model , Manufacturer and Year with ROLLUP

In [19]:
#Equivalent GROUP BY GROUPING SETS 
# ((model,manufacturer,year), (model,manufacturer),(model), ())
sql_query = "SELECT model,manufacturer,year,COUNT(1) \
FROM used_cars \
WHERE model = 'camry' \
GROUP BY model,manufacturer,year \
WITH ROLLUP ORDER BY COUNT(1) DESC"


sqlDF = spark.sql(sql_query)
sqlDF.show(100,truncate = False)

+-----+------------+----+--------+
|model|manufacturer|year|count(1)|
+-----+------------+----+--------+
|null |null        |null|2827    |
|camry|null        |null|2827    |
|camry|toyota      |null|2825    |
|camry|toyota      |2007|253     |
|camry|toyota      |2009|199     |
|camry|toyota      |2018|194     |
|camry|toyota      |2011|188     |
|camry|toyota      |2012|186     |
|camry|toyota      |2019|143     |
|camry|toyota      |2014|137     |
|camry|toyota      |2017|134     |
|camry|toyota      |2015|132     |
|camry|toyota      |2008|126     |
|camry|toyota      |2016|118     |
|camry|toyota      |2005|117     |
|camry|toyota      |2013|115     |
|camry|toyota      |2010|110     |
|camry|toyota      |2004|103     |
|camry|toyota      |2003|87      |
|camry|toyota      |2002|83      |
|camry|toyota      |2006|72      |
|camry|toyota      |2000|71      |
|camry|toyota      |2001|54      |
|camry|toyota      |1999|50      |
|camry|toyota      |2020|41      |
|camry|toyota      |

In [20]:
sql_query = "SELECT model, COUNT(1) \
FROM used_cars \
WHERE model = 'camry' \
GROUP BY model" 


sqlDF = spark.sql(sql_query)
sqlDF.show(28,truncate = False)

+-----+--------+
|model|count(1)|
+-----+--------+
|camry|2827    |
+-----+--------+



In [21]:
sql_query = "SELECT model, manufacturer ,COUNT(1) \
FROM used_cars \
WHERE model = 'camry' \
GROUP BY model , manufacturer" 


sqlDF = spark.sql(sql_query)
sqlDF.show(28,truncate = False)

+-----+------------+--------+
|model|manufacturer|count(1)|
+-----+------------+--------+
|camry|toyota      |2825    |
|camry|nissan      |2       |
+-----+------------+--------+



In [22]:
sql_query = "SELECT model, manufacturer ,year ,COUNT(1) \
FROM used_cars \
WHERE model = 'camry' \
GROUP BY model , manufacturer , year" 


sqlDF = spark.sql(sql_query)
sqlDF.show(100,truncate = False)

+-----+------------+----+--------+
|model|manufacturer|year|count(1)|
+-----+------------+----+--------+
|camry|toyota      |2018|194     |
|camry|toyota      |2019|143     |
|camry|toyota      |1999|50      |
|camry|toyota      |2005|117     |
|camry|toyota      |2016|118     |
|camry|toyota      |2006|72      |
|camry|toyota      |2020|41      |
|camry|toyota      |2010|110     |
|camry|toyota      |2001|54      |
|camry|toyota      |2015|132     |
|camry|toyota      |1996|20      |
|camry|toyota      |1997|27      |
|camry|toyota      |2009|199     |
|camry|toyota      |2004|103     |
|camry|toyota      |2011|188     |
|camry|toyota      |2012|186     |
|camry|toyota      |1998|35      |
|camry|toyota      |2003|87      |
|camry|toyota      |2007|253     |
|camry|toyota      |2013|115     |
|camry|toyota      |2014|137     |
|camry|toyota      |2017|134     |
|camry|toyota      |2008|126     |
|camry|toyota      |2002|83      |
|camry|toyota      |1993|5       |
|camry|toyota      |

# Count per Group : Model , Manufacturer and Year WITH CUBE

In [23]:
#Equivalent GROUP BY GROUPING SETS 
# (model,manufacturer,year)
# (model,manufacturer)
# (manufacturer,year)
# (model,year)
# (model)
# (manufacturer)
# (year)
# ()


sql_query = "SELECT model,manufacturer,year,COUNT(1) \
FROM used_cars \
GROUP BY model,manufacturer,year \
WITH CUBE ORDER BY COUNT(1) DESC"


sqlDF = spark.sql(sql_query)
sqlDF.show(25,truncate = False)



+-----+------------+----+--------+
|model|manufacturer|year|count(1)|
+-----+------------+----+--------+
|null |null        |null|393985  |
|null |ford        |null|64855   |
|null |chevrolet   |null|50643   |
|null |null        |2018|32644   |
|null |null        |2017|32544   |
|null |toyota      |null|31600   |
|null |null        |2013|28198   |
|null |null        |2015|27983   |
|null |null        |2016|27401   |
|null |null        |2014|26991   |
|null |null        |2019|22889   |
|null |null        |2012|22334   |
|null |honda       |null|19867   |
|null |null        |2011|19020   |
|null |null        |2020|17807   |
|null |jeep        |null|17461   |
|null |nissan      |null|17395   |
|null |ram         |null|16464   |
|null |null        |2008|16281   |
|null |null        |null|16185   |
|null |gmc         |null|15431   |
|null |null        |2010|14985   |
|null |null        |2007|14181   |
|null |bmw         |null|13745   |
|null |dodge       |null|12369   |
+-----+------------+

# DISTINCT Model

In [24]:
sql_query = "SELECT COUNT(DISTINCT(model)) FROM used_cars "
sqlDF = spark.sql(sql_query)
sqlDF.show(25,truncate = False)

+---------------------+
|count(DISTINCT model)|
+---------------------+
|28281                |
+---------------------+



# APPROX_COUNT_DISTINCT Model

In [25]:
sql_query = "SELECT APPROX_COUNT_DISTINCT(model) FROM used_cars "
sqlDF = spark.sql(sql_query)
sqlDF.show(25,truncate = False)


+----------------------------+
|approx_count_distinct(model)|
+----------------------------+
|28426                       |
+----------------------------+



# DENSE RANK by Price

In [26]:
sql_query = "SELECT model,manufacturer,year, \
price, \
DENSE_RANK () OVER ( \
ORDER BY price DESC \
) price_rank \
FROM used_cars "

sqlDF = spark.sql(sql_query)
sqlDF.show(25,truncate = False)

21/11/26 15:12:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
21/11/26 15:12:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+----------------------+-------------+----+----------+----------+
|model                 |manufacturer |year|price     |price_rank|
+----------------------+-------------+----+----------+----------+
|tundra                |toyota       |2007|3736928711|1         |
|4runner               |toyota       |1999|3736928711|1         |
|benz e320             |mercedes-benz|2000|3024942282|2         |
|benz s430             |mercedes-benz|2000|3024942282|2         |
|null                  |chevrolet    |2021|3009548743|3         |
|wrangler              |jeep         |1989|1410065407|4         |
|vnl                   |volvo        |2006|1234567890|5         |
|f350 super duty lariat|ford         |1999|1111111111|6         |
|null                  |jeep         |2020|1111111111|6         |
|null                  |chevrolet    |1960|987654321 |7         |
|null                  |chevrolet    |1960|987654321 |7         |
|titan se kingcab      |nissan       |2008|135008900 |8         |
|project  

# DENSE RANK Order by Price Partition by manufacturer

In [29]:
sql_query = "SELECT model,manufacturer,year,price, \
DENSE_RANK () OVER ( \
PARTITION BY manufacturer \
ORDER BY price DESC \
) price_rank \
FROM used_cars "

sqlDF = spark.sql(sql_query)
sqlDF.show(25,truncate = False)

+---------------------------+------------+----+---------+----------+
|model                      |manufacturer|year|price    |price_rank|
+---------------------------+------------+----+---------+----------+
|Chevy/gmc                  |null        |2003|123456789|1         |
|Rare Car                   |null        |2021|123456789|1         |
|all                        |null        |2021|5000000  |2         |
|any and all                |null        |2002|3226714  |3         |
|Recycling                  |null        |1902|1666666  |4         |
|Any make any model         |null        |2022|1000000  |5         |
|Call for a free estimate   |null        |2019|1000000  |5         |
|Any make any model         |null        |2022|1000000  |5         |
|Any make any model         |null        |2022|1000000  |5         |
|f250 xlt crewcab           |null        |2019|554900   |6         |
|Lambo Aventador SV Roadster|null        |2016|516999   |7         |
|hINO 268                   |null 

In [28]:
sql_query = "SELECT * FROM ( SELECT model,manufacturer,year,price, \
DENSE_RANK () OVER ( \
PARTITION BY manufacturer \
ORDER BY price DESC \
) price_rank \
FROM used_cars ) t \
WHERE price_rank < 4 "

sqlDF = spark.sql(sql_query)
sqlDF.show(25,truncate = False)

+-------------------+------------+----+---------+----------+
|model              |manufacturer|year|price    |price_rank|
+-------------------+------------+----+---------+----------+
|Chevy/gmc          |null        |2003|123456789|1         |
|Rare Car           |null        |2021|123456789|1         |
|all                |null        |2021|5000000  |2         |
|any and all        |null        |2002|3226714  |3         |
|regal              |buick       |1999|123456789|1         |
|lacrosse           |buick       |2013|123456   |2         |
|skylark convertible|buick       |1953|110000   |3         |
|xk 150             |jaguar      |1959|150000   |1         |
|xk 150 se dhc      |jaguar      |1959|150000   |1         |
|xk150 se dhc       |jaguar      |1959|150000   |1         |
|xk150 se dhc       |jaguar      |1959|150000   |1         |
|xke                |jaguar      |1971|75000    |2         |
|null               |jaguar      |1970|65000    |3         |
|ipace              |jag